# 

1. Descuentos por volumen | Utilización según tipo de negocio

En este trabajo nos proponemos caracterizar la forma en que los distintos negocios utilizan los descuentos por volumen. La pregunta de investigación es: ¿los distintos tipos de negocios utilizan de forma distinta los descuentos por volumen? 

Con el término 'la forma de utilización de los descuentos' nos referimos al tipo de producto comprado bajo un descuento de volumen, la cantidad de unidades compradas en promedio y el descuento particular utilizado.

Nuestras hipótesis pueden expresarse como:

- GENERAL: Los distintos tipos de cliente se diferencian en la forma de utilizar los descuentos por volumen
    
- El tipo de cliente influye en el promedio de unidades compradas bajo una oferta de volumen 
    
- El tipo de cliente influye en el descuento particular utilizado de las ofertas de volumen
    
- El tipo de cliente influye en la proporción de líneas con descuento respecto al total de líneas
    
- El tipo de cliente influye en la categoría de producto comprado bajo una oferta por volumen
    
- Nos enfocaremos en:
    

1. Caracterizar a los tipos de clientes de la empresa y su comportamiento de compra
2. Caracterizar a los descuentos por volumen (qué productos los aplican, cuántas unidades vendidas acumulan, promedio de unidades las líneas de las ordenes que los utilizan)
3. Caracterizar la relación entre los distintos tipos de clientes y los descuentos por volumen (qué tipos de productos compran, cuántas unidades compran en promedio y qué descuento es el que más utilizan)

1\. Caracterizar a los tipos de clientes de la empresa y su comportamiento de compra

In [66]:
WITH CTE AS (
    SELECT r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    --inner porque solo nos interesan las ventas con un reseller asociado
    INNER JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
)

SELECT DISTINCT BusinessType, volProm,

--lines
COUNT(LineId) OVER(PARTITION BY BusinessType, volProm) 
AS BusinessPromNumLines,

COUNT(LineId) OVER(PARTITION BY BusinessType) 
AS BusinessNumLines,

COUNT(LineId) OVER(PARTITION BY BusinessType, volProm) * 1.0 / (COUNT(LineId) OVER(PARTITION BY BusinessType)) 
AS percBusinessPromLines,

COUNT(LineId) OVER(PARTITION BY BusinessType) * 1.0 / (COUNT(LineId) OVER())
AS percBusinessNumLines,

--------
--units
SUM(OrderQuantity) OVER(PARTITION BY BusinessType, VolProm) 
AS BusinessPromNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType) 
AS BusinessNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm)* 1.0 / (SUM(OrderQuantity) OVER(PARTITION BY BusinessType))
AS percBusinessPromNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType)* 1.0 / (SUM(OrderQuantity) OVER())
AS percBusinessNumUnits,

AVG(OrderQuantity * 1.0) OVER(PARTITION BY BusinessType, volProm) 
AS avgBusinessPromOrderQty,

AVG(OrderQuantity * 1.0) OVER(PARTITION BY BusinessType) 
AS avgBusinessOrderQty


FROM CTE
ORDER BY BusinessType, volProm

(6 rows affected)

Total execution time: 00:00:01.349

BusinessType,volProm,BusinessPromNumLines,BusinessNumLines,percBusinessPromLines,percBusinessNumLines,BusinessPromNumUnits,BusinessNumUnits,percBusinessPromNumUnits,percBusinessNumUnits,avgBusinessPromOrderQty,avgBusinessOrderQty
Specialty Bike Shop,noVolProm,7909,7987,0.990234130462,0.131246405389,20914,22076,0.947363652835,0.102976984578,2.644329,2.763991
Specialty Bike Shop,VolProm,78,7987,0.009765869537,0.131246405389,1162,22076,0.052636347164,0.102976984578,14.897435,2.763991
Value Added Reseller,noVolProm,24698,25134,0.982652980027,0.413014542765,73027,80309,0.909325231294,0.374613999570,2.956798,3.195233
Value Added Reseller,VolProm,436,25134,0.017347019972,0.413014542765,7282,80309,0.090674768705,0.374613999570,16.701834,3.195233
Warehouse,noVolProm,26250,27734,0.946491670873,0.455739051844,91255,111993,0.814827712446,0.522409015850,3.476380,4.038112
Warehouse,VolProm,1484,27734,0.053508329126,0.455739051844,20738,111993,0.185172287553,0.522409015850,13.974393,4.038112


In [67]:
WITH CTE AS (
    SELECT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
    r.BusinessType, SalesOrderNumber, PromotionKey,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
    ON rs.ProductKey = dp.ProductKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
    ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
    ON ps.ProductCategoryKey = pc.ProductCategoryKey
)

SELECT DISTINCT BusinessType, PromotionKey,

--lines
COUNT(LineId) OVER(PARTITION BY BusinessType, PromotionKey) 
AS BusinessPromKeyNumLines,

COUNT(LineId) OVER(PARTITION BY BusinessType) 
AS BusinessNumLines,

COUNT(LineId) OVER(PARTITION BY BusinessType, PromotionKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType))
AS percBusinessPromKeyLines,

COUNT(LineId) OVER(PARTITION BY BusinessType) * 1.0 /  (COUNT(LineId) OVER())
AS percBusinessNumLines,

-----

--units
SUM(OrderQuantity) OVER(PARTITION BY BusinessType, PromotionKey)
AS BusinessPromKeyNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType) 
AS BusinessNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, PromotionKey)* 1.0 /  SUM(OrderQuantity) OVER(PARTITION BY BusinessType)
AS percBusinessPromKeyNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType) * 1.0 /  (SUM(OrderQuantity) OVER())
AS percBusinessNumUnits,

AVG(OrderQuantity*1.0) OVER(PARTITION BY BusinessType, PromotionKey) 
AS avgBusinessPromKeyOrderQty



FROM CTE
ORDER BY BusinessType, PromotionKey

(33 rows affected)

Total execution time: 00:00:01.038

BusinessType,PromotionKey,BusinessPromKeyNumLines,BusinessNumLines,percBusinessPromKeyLines,percBusinessNumLines,BusinessPromKeyNumUnits,BusinessNumUnits,percBusinessPromKeyNumUnits,percBusinessNumUnits,avgBusinessPromKeyOrderQty
Specialty Bike Shop,1,7662,7987,0.959308876925,0.131246405389,20431,22076,0.925484689255,0.102976984578,2.666536
Specialty Bike Shop,2,43,7987,0.005383748591,0.131246405389,523,22076,0.023690886030,0.102976984578,12.162790
Specialty Bike Shop,3,33,7987,0.004131714035,0.131246405389,584,22076,0.026454067765,0.102976984578,17.696969
Specialty Bike Shop,4,2,7987,0.000250406911,0.131246405389,55,22076,0.002491393368,0.102976984578,27.500000
Specialty Bike Shop,7,35,7987,0.004382120946,0.131246405389,55,22076,0.002491393368,0.102976984578,1.571428
Specialty Bike Shop,8,31,7987,0.003881307124,0.131246405389,91,22076,0.004122123573,0.102976984578,2.935483
Specialty Bike Shop,9,23,7987,0.002879679479,0.131246405389,55,22076,0.002491393368,0.102976984578,2.391304
Specialty Bike Shop,11,24,7987,0.003004882934,0.131246405389,73,22076,0.003306758470,0.102976984578,3.041666
Specialty Bike Shop,13,89,7987,0.011143107549,0.131246405389,132,22076,0.005979344084,0.102976984578,1.483146
Specialty Bike Shop,14,45,7987,0.005634155502,0.131246405389,77,22076,0.003487950715,0.102976984578,1.711111


Chequeamos que todas ventas tengan un reseller asociado:

In [37]:
SELECT r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
    WHERE r.ResellerKey IS NULL

(0 rows affected)

Total execution time: 00:00:00.076

BusinessType,SalesOrderNumber,LineId,volProm,OrderQuantity


2\. Caracterizar a los descuentos por volumen (qué productos los aplican, cuántas unidades vendidas acumulan, promedio de unidades las líneas de las ordenes que los utilizan)

In [34]:
WITH CTE AS(
SELECT DISTINCT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm

FROM AdventureWorksDW2019.dbo.FactResellerSales rs
LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
ON rs.ProductKey = dp.ProductKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
ON ps.ProductCategoryKey = pc.ProductCategoryKey

)

SELECT DISTINCT EnglishProductCategoryName, EnglishProductSubcategoryName, ProductKey, volProm,
-- acá tendría que ser un count distinct pero eso no se permite en las window functions, rehacer de otra manera
COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey, volProm)  AS prodQty,
FORMAT(COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey, volProm) * 1.0 / COUNT(ProductKey) OVER(PARTITION BY ProductSubcategoryKey),'P')  AS percProd

FROM CTE
ORDER BY ProductKey

(473 rows affected)

Total execution time: 00:00:00.466

EnglishProductCategoryName,EnglishProductSubcategoryName,ProductKey,volProm,prodQty,percProd
Accessories,Helmets,212,noVolProm,9,52.94%
Accessories,Helmets,213,noVolProm,9,52.94%
Accessories,Helmets,213,VolProm,8,47.06%
Accessories,Helmets,214,noVolProm,9,52.94%
Accessories,Helmets,214,VolProm,8,47.06%
Accessories,Helmets,215,noVolProm,9,52.94%
Accessories,Helmets,215,VolProm,8,47.06%
Accessories,Helmets,216,noVolProm,9,52.94%
Accessories,Helmets,216,VolProm,8,47.06%
Accessories,Helmets,217,noVolProm,9,52.94%


In [42]:
SELECT DISTINCT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,PromotionKey
--CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm

FROM AdventureWorksDW2019.dbo.FactResellerSales rs
LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
ON rs.ProductKey = dp.ProductKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
ON ps.ProductCategoryKey = pc.ProductCategoryKey

ORDER BY ProductKey

(588 rows affected)

Total execution time: 00:00:00.209

ProductKey,ProductCategoryKey,EnglishProductCategoryName,ProductSubcategoryKey,EnglishProductSubcategoryName,PromotionKey
212,4,Accessories,31,Helmets,1
213,4,Accessories,31,Helmets,1
213,4,Accessories,31,Helmets,2
213,4,Accessories,31,Helmets,3
213,4,Accessories,31,Helmets,8
214,4,Accessories,31,Helmets,1
214,4,Accessories,31,Helmets,2
214,4,Accessories,31,Helmets,3
214,4,Accessories,31,Helmets,11
215,4,Accessories,31,Helmets,1


Productos en promo

In [54]:
SELECT DISTINCT rs.ProductKey, PromotionKey, DiscountAmount, OrderDate, OrderQuantity,
CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm
FROM AdventureWorksDW2019.dbo.FactResellerSales rs
WHERE ProductKey = 213
ORDER BY OrderDate, ProductKey




(112 rows affected)

Total execution time: 00:00:00.033

ProductKey,PromotionKey,DiscountAmount,OrderDate,OrderQuantity,volProm
213,8,"1,6822",2011-12-29 00:00:00.000,1,noVolProm
213,8,"3,3644",2011-12-29 00:00:00.000,2,noVolProm
213,8,"5,0466",2011-12-29 00:00:00.000,3,noVolProm
213,8,"6,7288",2011-12-29 00:00:00.000,4,noVolProm
213,8,"8,4111",2011-12-29 00:00:00.000,5,noVolProm
213,8,"10,0933",2011-12-29 00:00:00.000,6,noVolProm
213,8,"13,4577",2011-12-29 00:00:00.000,8,noVolProm
213,8,"16,8221",2011-12-29 00:00:00.000,10,noVolProm
213,1,0,2012-01-29 00:00:00.000,1,noVolProm
213,1,0,2012-01-29 00:00:00.000,2,noVolProm


In [55]:
WITH CTE AS (SELECT rs.ProductKey, CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm
FROM AdventureWorksDW2019.dbo.FactResellerSales rs)
SELECT DISTINCT CTE.ProductKey
FROM CTE
WHERE volProm = 'VolProm'


(139 rows affected)

Total execution time: 00:00:00.067

ProductKey
355
215
524
384
232
576
361
564
315
470


In [47]:
SELECT *
FROM AdventureWorksDW2019.dbo.DimPromotion


(16 rows affected)

Total execution time: 00:00:00.064

PromotionKey,PromotionAlternateKey,EnglishPromotionName,SpanishPromotionName,FrenchPromotionName,DiscountPct,EnglishPromotionType,SpanishPromotionType,FrenchPromotionType,EnglishPromotionCategory,SpanishPromotionCategory,FrenchPromotionCategory,StartDate,EndDate,MinQty,MaxQty
1,1,No Discount,Sin descuento,Aucune remise,0,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29 00:00:00.000,2014-06-30 00:00:00.000,0,NULL
2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),"0,02",Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00.000,2013-12-28 00:00:00.000,11,14
3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),"0,05",Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00.000,2013-12-28 00:00:00.000,15,24
4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),"0,1",Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00.000,2013-12-28 00:00:00.000,25,40
5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),"0,15",Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00.000,2013-12-28 00:00:00.000,41,60
6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),"0,2",Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29 00:00:00.000,2013-12-28 00:00:00.000,61,NULL
7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,"0,35",Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12 00:00:00.000,2011-12-28 00:00:00.000,0,NULL
8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,"0,1",Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29 00:00:00.000,2012-01-28 00:00:00.000,0,NULL
9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,"0,3",Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29 00:00:00.000,2012-02-28 00:00:00.000,0,NULL
10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,"0,5",Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12 00:00:00.000,2013-02-26 00:00:00.000,0,NULL


3\. Caracterizar la relación entre los distintos tipos de clientes y los descuentos por volumen (qué tipos de productos compran, cuántas unidades compran en promedio y qué descuento es el que más utilizan)

In [68]:
WITH CTE AS (
    SELECT rs.ProductKey, pc.ProductCategoryKey, pc.EnglishProductCategoryName, ps.ProductSubcategoryKey, ps.EnglishProductSubcategoryName,
    r.BusinessType, SalesOrderNumber,
    CONCAT(SalesOrderNumber, '-', SalesOrderLineNumber) AS LineId, 
    CASE WHEN PromotionKey IN(2,3,4,5,6) THEN 'VolProm' ELSE 'noVolProm' END volProm,
    OrderQuantity

    FROM AdventureWorksDW2019.dbo.FactResellerSales rs
    LEFT JOIN AdventureWorksDW2019.dbo.DimReseller r
    ON rs.ResellerKey = r.ResellerKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProduct dp
    ON rs.ProductKey = dp.ProductKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductSubcategory ps
    ON dp.ProductSubcategoryKey = ps.ProductSubcategoryKey
    LEFT JOIN AdventureWorksDW2019.dbo.DimProductCategory pc 
    ON ps.ProductCategoryKey = pc.ProductCategoryKey
)

SELECT DISTINCT BusinessType, volProm, EnglishProductCategoryName,

--lines
COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey)
AS BusinessCatPromNumLines,

COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey)
AS BusinessCatLines,

COUNT(LineId) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey))
AS percBusinessCatPromLines,

COUNT(LineId) OVER(PARTITION BY BusinessType, ProductCategoryKey) * 1.0 /  (COUNT(LineId) OVER(PARTITION BY BusinessType))
AS percBusinessCatLines,


--units
SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) 
AS BusinessCatPromNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, ProductCategoryKey) 
AS BusinessCatNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey)* 1.0 /  SUM(OrderQuantity) OVER(PARTITION BY BusinessType, ProductCategoryKey)
AS percBusinessCatPromNumUnits,

SUM(OrderQuantity) OVER(PARTITION BY BusinessType, ProductCategoryKey) * 1.0 /  (SUM(OrderQuantity) OVER(PARTITION BY BusinessType))
AS percBusinessCatNumUnits,

AVG(OrderQuantity*1.0) OVER(PARTITION BY BusinessType, volProm, ProductCategoryKey) 
AS avgBusinessPromCatOrderQty



FROM CTE
ORDER BY BusinessType, EnglishProductCategoryName, volProm

(21 rows affected)

Total execution time: 00:00:01.295

BusinessType,volProm,EnglishProductCategoryName,BusinessCatPromNumLines,BusinessCatLines,percBusinessCatPromLines,percBusinessCatLines,BusinessCatPromNumUnits,BusinessCatNumUnits,percBusinessCatPromNumUnits,percBusinessCatNumUnits,avgBusinessPromCatOrderQty
Specialty Bike Shop,noVolProm,Accessories,749,749,1.000000000000,0.093777388255,2392,2392,1.000000000000,0.108352962493,3.193591
Specialty Bike Shop,noVolProm,Bikes,3427,3427,1.000000000000,0.429072242393,7094,7094,1.000000000000,0.321344446457,2.070032
Specialty Bike Shop,noVolProm,Clothing,2580,2658,0.970654627539,0.332790785025,8857,10019,0.884020361313,0.453841275593,3.432945
Specialty Bike Shop,VolProm,Clothing,78,2658,0.029345372460,0.332790785025,1162,10019,0.115979638686,0.453841275593,14.897435
Specialty Bike Shop,noVolProm,Components,1153,1153,1.000000000000,0.144359584324,2571,2571,1.000000000000,0.116461315455,2.229835
Value Added Reseller,noVolProm,Accessories,1946,1955,0.995396419437,0.077783082676,7662,7776,0.985339506172,0.096826009538,3.937307
Value Added Reseller,VolProm,Accessories,9,1955,0.004603580562,0.077783082676,114,7776,0.014660493827,0.096826009538,12.666666
Value Added Reseller,noVolProm,Bikes,12319,12355,0.997086199919,0.491565210471,34077,34509,0.987481526558,0.429702773039,2.766214
Value Added Reseller,VolProm,Bikes,36,12355,0.002913800080,0.491565210471,432,34509,0.012518473441,0.429702773039,12.000000
Value Added Reseller,noVolProm,Clothing,3672,4057,0.905102292334,0.161414816583,15125,21787,0.694221324643,0.271289643751,4.119008
